In [1]:
import pybullet as p
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import time
import pybullet_data
import serial

In [2]:
WayPoints = []
stepSize = 125
numMoveableJoints = 6
TimeStep = 1./120.
ser = serial.Serial("COM8", 9600, timeout = 1)

physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
planeId = p.loadURDF("plane.urdf")
cubeStartPos = [0,0,0]
cubeStartOrientation = p.getQuaternionFromEuler([0,0,0])
robot = p.loadURDF("C:/Users/kenne/Downloads/kuka_experimental/kuka_kr16_support/urdf/kr16_2.urdf",cubeStartPos, cubeStartOrientation, useFixedBase = 1)
print([j[0] for j in p.getJointStates(robot,range(8))])
p.setRealTimeSimulation(0)
p.setGravity(0,0,-9.8)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [3]:
p.setRealTimeSimulation(0)

In [4]:
def moveTo(ori, loc):
    orientation = p.getQuaternionFromEuler(ori)
    tartgetPositionJoints = p.calculateInverseKinematics(robot,numMoveableJoints,loc, targetOrientation = orientation)
    p.setJointMotorControlArray(robot, range(numMoveableJoints), p.POSITION_CONTROL,targetPositions= tartgetPositionJoints)

In [5]:
def moveAndSim(ori =[0,0,0], loc = [0,0,0]):
    moveTo(ori, loc)
    for i in range (stepSize):
        p.stepSimulation()
        time.sleep(TimeStep)

In [6]:
def radToDeg(x):
    return (x*180/3.14)

def moveRobotViaSerial(SimulatedRobot):
    sleepTime = 2
    BaseAngle = radToDeg(p.getJointStates(SimulatedRobot,[0])[0][0]) + 90
    ShoulderAngle = abs(radToDeg(p.getJointStates(SimulatedRobot,[1])[0][0]) +180)
    ElbowAngle = -(radToDeg(p.getJointStates(SimulatedRobot,[2])[0][0])) + 180
    WristAngle = radToDeg(p.getJointStates(SimulatedRobot,[3])[0][0]) + 90
    print("Base Angle | ", BaseAngle,"\nShoulderAngle | ", ShoulderAngle, "\nElbowAngle | ", ElbowAngle,"\nWristAngle | ", WristAngle)
    ser.write(bytes("Servo4 " + str(int(BaseAngle)), 'ascii'))
    time.sleep(sleepTime)
    ser.write(bytes("Servo3 " + str(int(ShoulderAngle)), 'ascii'))
    time.sleep(sleepTime)
    ser.write(bytes("Servo2 " + str(int(ElbowAngle)), 'ascii'))
    time.sleep(sleepTime)
    ser.write(bytes("Servo1 " + str(int(WristAngle)), 'ascii'))
    time.sleep(sleepTime)
    data = ser.read(88)
    print(data)

In [7]:
print("Base Angle in radian" , p.getJointStates(robot,[0])[0][0])
print("Shoulder Angle in radian" , p.getJointStates(robot,[1])[0][0])
print("Elbow Angle in radian" , p.getJointStates(robot,[2])[0][0])
print("Wrist Angle in radian" , p.getJointStates(robot,[3])[0][0])

Base Angle in radian 0.0
Shoulder Angle in radian 0.0
Elbow Angle in radian 0.0
Wrist Angle in radian 0.0


In [8]:
GlobalPosition = [0,0,.5]
GlobalOrientation = [0,0,0]
def setx(x):
    GlobalPosition[0] = x
def sety(y):
    GlobalPosition[1] = y
def setz(z):
    GlobalPosition[2] = z
    
def setThetaX(ThetaX):
    GlobalOrientation[0] = ThetaX * 3.14 / 180
def setThetaY(WristAngle):
    GlobalOrientation[1] = WristAngle * 3.14 / 180
def setThetaZ(ThetaZ):
    GlobalOrientation[2] = ThetaZ * 3.14 / 180

interact(setx, x=widgets.FloatSlider(min=-2, max=2, step=.05, value=1));
interact(sety, y=widgets.FloatSlider(min=-2, max=2, step=.05, value=0));
interact(setz, z=widgets.FloatSlider(min=0, max=3, step=.05, value=0.5));


#interact(setThetaX, ThetaX=widgets.IntSlider(min=-90, max=90, step=1, value=0));
interact(setThetaY, WristAngle=widgets.IntSlider(min=-180, max=180, step=1, value=90));
#interact(setThetaZ, ThetaZ=widgets.IntSlider(min=-90, max=90, step=1, value=0));

SimulateAndMove = widgets.Button(description="Move Simulation")
display(SimulateAndMove)
def MoveAndSim(b):
    moveAndSim(ori = GlobalOrientation, loc = GlobalPosition)
    #print(GlobalPosition)
SimulateAndMove.on_click(MoveAndSim)

SimulateAndMoveRobot = widgets.Button(description="Move Robot")
display(SimulateAndMoveRobot)
def MoveAndSimRobot(b):
    moveRobotViaSerial(robot)
    #print(GlobalPosition)
SimulateAndMoveRobot.on_click(MoveAndSimRobot)

""" 
def helperSaveWaypoint(x):
    WayPoints = WayPoints.append(x)
    
SaveWaypoint = widgets.Button(description="Save Waypoint")
display(SaveWaypoint)
def saveWayPoint(b):
    #WayPoints = WayPoints.append([GlobalOrientation, GlobalPosition])
    helperSaveWaypoint([GlobalOrientation, GlobalPosition])
    print("Waypoint Saved", [GlobalOrientation, GlobalPosition])
SaveWaypoint.on_click(saveWayPoint)
"""

interactive(children=(FloatSlider(value=1.0, description='x', max=2.0, min=-2.0, step=0.05), Output()), _dom_c…

interactive(children=(FloatSlider(value=0.0, description='y', max=2.0, min=-2.0, step=0.05), Output()), _dom_c…

interactive(children=(FloatSlider(value=0.5, description='z', max=3.0, step=0.05), Output()), _dom_classes=('w…

interactive(children=(IntSlider(value=90, description='WristAngle', max=180, min=-180), Output()), _dom_classe…

Button(description='Move Simulation', style=ButtonStyle())

Button(description='Move Robot', style=ButtonStyle())

' \ndef helperSaveWaypoint(x):\n    WayPoints = WayPoints.append(x)\n    \nSaveWaypoint = widgets.Button(description="Save Waypoint")\ndisplay(SaveWaypoint)\ndef saveWayPoint(b):\n    #WayPoints = WayPoints.append([GlobalOrientation, GlobalPosition])\n    helperSaveWaypoint([GlobalOrientation, GlobalPosition])\n    print("Waypoint Saved", [GlobalOrientation, GlobalPosition])\nSaveWaypoint.on_click(saveWayPoint)\n'

In [9]:
Start = widgets.Button(description="Move To Start Way Point")
display(Start)
Foward = widgets.Button(description="Foward Waypoint")
display(Foward)
Back = widgets.Button(description="Back Waypoint")
display(Back)
WaypointCounter = 0

def StartFunction(b):
    moveAndSim(WayPoints[0][0], WayPoints[0][1])
    print("Move To Start")
Start.on_click(StartFunction)

def ForwardFunction(b):
    WaypointCounter = WaypointCounter + 1
    moveAndSim(WayPoints[WaypointCounter][0], WayPoints[WaypointCounter][1])
    print("Way Point | ", WaypointCounter)
Start.on_click(StartFunction)

def BackFunction(b): 
    for i in range(len(WayPoints)):
        moveToWaypoint(WayPoints[i])
        print("Way Point | ", WayPoints[i])
Start.on_click(StartFunction)
Foward.on_click(ForwardFunction)
Back.on_click(BackFunction)

Button(description='Move To Start Way Point', style=ButtonStyle())

Button(description='Foward Waypoint', style=ButtonStyle())

Button(description='Back Waypoint', style=ButtonStyle())

In [10]:
print(WayPoints)

[]


In [7]:
buttonxdown = widgets.Button(description="x-")
buttonxplus = widgets.Button(description="x+")
buttonydown = widgets.Button(description="y-")
buttonyplus = widgets.Button(description="y+")
buttonzdown = widgets.Button(description="z-")
buttonzplus = widgets.Button(description="z+")
output = widgets.Output()
location = [1,0,.5]

display(buttonxplus)
def xup(b):
    location[0] = location[0] + .1
    moveAndSim(loc = location)
    print(location)

display(buttonxdown)
def xdown(b):
    location[0] = location[0] - .1
    moveAndSim(loc = location)
    print(location)
    
display(buttonyplus)
def yup(b):
    location[1] = location[1] + .1
    moveAndSim(loc = location)
    print(location)

display(buttonydown)
def ydown(b):
    location[1] = location[1] - .1
    moveAndSim(loc = location)
    print(location)

display(buttonzplus)
def zup(b):
    location[2] = location[2] + .1
    moveAndSim(loc = location)
    print(location)

display(buttonzdown)
def zdown(b):
    location[2] = location[2] - .1
    moveAndSim(loc = location)
    print(location)
    
    
buttonxplus.on_click(xup)
buttonxdown.on_click(xdown)
buttonyplus.on_click(yup)
buttonydown.on_click(ydown)
buttonzplus.on_click(zup)
buttonzdown.on_click(zdown)

Button(description='x+', style=ButtonStyle())

Button(description='x-', style=ButtonStyle())

Button(description='y+', style=ButtonStyle())

Button(description='y-', style=ButtonStyle())

Button(description='z+', style=ButtonStyle())

Button(description='z-', style=ButtonStyle())